In [ ]:
#importamos los paquetes que usaremos

import os,codecs,sys
import re,time,shutil
import numpy as np
import pandas as pd
import string, warnings
import sklearn

from sklearn import model_selection, preprocessing, linear_model, metrics, svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_selection import VarianceThreshold
from joblib import dump, load
from sklearn.base import clone
import torch

from datetime import datetime

In [ ]:
#instalamos la paquetería de transformers para poder usar Bert
!pip install transformers

In [ ]:
#declaramos los datos a analizar y las carpetas
tipo_datos = "sinURLs"
input_path= "/content/drive/My Drive/Bases SS/"+tipo_datos
input_pathT = "/content/drive/My Drive/Bases SS"
file = "Nuevo Sexo sinNada"
input_pathImp = "/content/drive/My Drive/Resultados SS/Bert/"
#!ls "/content/drive/My Drive/Resultados SS/"

In [ ]:
#Creamos acceso a google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#leemos la info
print ('Comenzamos \nLeemos el path: ', input_path)

with open(input_pathT+"/"+"truth.txt","r") as lf:
    truth = {}
    for line in lf.readlines():
        line = line.split(":::")   
        truth[line[0]]=[]        
        for item in line[1:]:            
            truth[line[0]].append(item.rstrip())
#####**** Guarda en una lista los txtfile, las etiquetas ****########       
txtfiles=[]
labels_gen=[]
labels_leng=[]

contador = 1
for item in os.listdir(input_path):
    if os.path.splitext(item)[1] == ".txt" and os.path.splitext(item)[0][:-1] in truth.keys():   
        with open(input_path+"/"+item) as fileLeer:
            txtfiles.append(fileLeer.read())
            labels_gen.append(truth[os.path.splitext(item)[0][:-1]][0])
            labels_leng.append(truth[os.path.splitext(item)[0][:-1]][1])
            if contador % 100 == 0:
              print(contador)
            contador = contador+1

Comenzamos 
Leemos el path:  /content/drive/My Drive/Bases SS/sinURLs
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200


In [ ]:
#declaramos nuestra función para evaluar

def Evaluacion(classifiers, X_train, y_train_label,pathGuardar, carpetaUsada, Tipolabels):
  #obtenemos la fecha
  from datetime import datetime
  import time
  now = datetime.now()
  dt_string = now.strftime("%d-%m-%Y %H-%M-%S")
  dt_stringPuntos = now.strftime("%d-%m-%Y %H:%M:%S")
  #declaramos el path para guardar el archivo junto con el nombre del file
  filepath = pathGuardar+"/BERT "+Tipolabels+" "+carpetaUsada+" "+dt_string+".txt"
  
  #comenzamos a escribir el archibo
  with open(filepath, "w") as output:
    
    #comenzamos a contar el tiempo
    start_time = time.time()
    #imprimimos datos relevantes para el txt
    output.write('Fecha de creación del archivo: '+str(dt_stringPuntos))
    output.write('\nCarpeta usada: '+str(carpetaUsada))
    output.write('\nPreprocesamiento de la info: '+str("BERT"))
    output.write('\nNúmero de archivos analizados: '+str(X_train.shape[0]))
    
    #For por cada uno de los algoritmos
    output.write('\n ---------------------------')
    warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

    ####*** guardan los resulados de cada una de las 50 iteraciones de RepeatedStratifiedKFold****######
    scores_rskf = []
    f1,precision,accuracy,recall = [],[],[],[]
    scores_rskfx = []
    f1x,precisionx,accuracyx,recallx = [],[],[],[]

    ######*** se Elijió RepeatedStratifiedKFold porque nos da una muestra representativa de c/u de las clases
    #### y así evitar sesgos
    rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5,random_state=36851234)
    i=0
    for train_index, test_index in rskf.split(X_train, y_train_label):
      #hacemos el split entre train y test
      X_train_foldsTxt, X_test_foldsTxt = X_train[train_index], X_train[test_index]
      y_train_folds, y_test_folds = y_train_label[train_index], y_train_label[test_index]
      
      #Entrenamos y predecimos
      i+=1
      print('*** Entrenando primer modelo ***',i)
      classifiers[0].fit(X_train_foldsTxt,y_train_folds)
      print('*** Predecir primer modelo ***',i)
      print("--- %s seconds ---" % (time.time() - start_time))
      y_pred = classifiers[0].predict(X_test_foldsTxt)
      n_correct = sum(y_pred == y_test_folds)
      print('*** Entrenando segundo modelo ***',i)
      classifiers[1].fit(X_train_foldsTxt,y_train_folds)
      print('*** Predecir segundo modelo ***',i)
      print("--- %s seconds ---" % (time.time() - start_time))
      y_predx = classifiers[1].predict(X_test_foldsTxt)
      n_correctx = sum(y_predx == y_test_folds)
      
      ########**** Métricas de evaluación ****#########
      #utilizamos la misma métrica, ya sea si predecimos sexo o variante de lenguaje, pero cambiamos el 
      #parametro binary por weighted
      if Tipolabels == "Género":
        scores_rskf.append(n_correct/len(y_pred))
        f1.append(metrics.f1_score(y_pred, y_test_folds, average='weighted', labels=np.unique(y_test_folds)))
        precision.append(metrics.precision_score(y_pred, y_test_folds,average='binary'))
        accuracy.append(metrics.accuracy_score(y_pred, y_test_folds))
        recall.append(metrics.recall_score(y_pred, y_test_folds,average='binary'))
        
        scores_rskfx.append(n_correctx/len(y_predx))
        f1x.append(metrics.f1_score(y_predx, y_test_folds, average='weighted', labels=np.unique(y_test_folds)))
        precisionx.append(metrics.precision_score(y_predx, y_test_folds,average='binary'))
        accuracyx.append(metrics.accuracy_score(y_predx, y_test_folds))
        recallx.append(metrics.recall_score(y_predx, y_test_folds,average='binary'))
      else:
        scores_rskf.append(n_correct/len(y_pred))
        f1.append(metrics.f1_score(y_pred, y_test_folds, average='weighted', labels=np.unique(y_test_folds)))
        precision.append(metrics.precision_score(y_pred, y_test_folds,average='weighted'))
        accuracy.append(metrics.accuracy_score(y_pred, y_test_folds))
        recall.append(metrics.recall_score(y_pred, y_test_folds,average='weighted'))
        
        scores_rskfx.append(n_correctx/len(y_predx))
        f1x.append(metrics.f1_score(y_predx, y_test_folds, average='weighted', labels=np.unique(y_test_folds)))
        precisionx.append(metrics.precision_score(y_predx, y_test_folds,average='weighted'))
        accuracyx.append(metrics.accuracy_score(y_predx, y_test_folds))
        recallx.append(metrics.recall_score(y_predx, y_test_folds,average='weighted'))
              
      print("*** Termino el entrenamiento de la capa ",i," ***")
      print("--- %s seconds ---" % (time.time() - start_time))
    
    ##############*** mean y std de los scores obtenidos de  RepeatedStratifiedKFold***###################
    mean_score = np.mean(scores_rskf)
    standar_score =np.std(scores_rskf)
    
    mean_scorex = np.mean(scores_rskfx)
    standar_scorex =np.std(scores_rskfx)
        
    output.write('\n *** '+str(classifiers[0])+'\n ---------------------- ')
    output.write('\n ---------Validación RepeatedStratifiedKFold------------------')
    output.write('\n  Scores:' + str(scores_rskf))
    output.write('\n  Mean:' + str(mean_score))
    output.write('\n  Standard deviation:' + str(standar_score))
    output.write('\n  F1:  '+ str(np.mean(f1)))
    output.write('\n  precision:  ' + str(np.mean(precision)))
    output.write('\n  recall:  ' + str(np.mean(recall)))
    output.write('\n  exactitud (accuracy):  ' + str(np.mean(accuracy)))
    output.write('\n  elapsed time:' + str(time.time() - start_time) )
    output.write('\n  #características:' + str(X_train.shape))
    
    output.write('\n \n ---------------------------')
    output.write('\n *** '+str(classifiers[1])+'\n ---------------------- ')
    output.write('\n ---------Validación RepeatedStratifiedKFold------------------')
    output.write('\n  Scores:' + str(scores_rskfx))
    output.write('\n  Mean:' + str(mean_scorex))
    output.write('\n  Standard deviation:' + str(standar_scorex))
    output.write('\n  F1:  '+ str(np.mean(f1x)))
    output.write('\n  precision:  ' + str(np.mean(precisionx)))
    output.write('\n  recall:  ' + str(np.mean(recallx)))
    output.write('\n  exactitud (accuracy):  ' + str(np.mean(accuracyx)))
    output.write('\n  elapsed time:' + str(time.time() - start_time) )
    output.write('\n  #características:' + str(X_train.shape))


In [ ]:
######*** Crea el dataframe y guarda el texto y las etiquetas ***######
trainDF = pd.DataFrame()
trainDF['text'] = txtfiles
trainDF['labels_gen'] = labels_gen
trainDF['labels_leng']=labels_leng
print(trainDF.head())
print("\nEl shape de nuestro dataframe es: ",trainDF.shape)

                                                text labels_gen labels_leng
0  ¡Recursos Didácticos Interactivos está disponi...       male   argentina
1  Cada día me encantas un poco mas 😍😍 12 😍😍\nOla...     female    colombia
2  Quiero ver quienes están en las malas\nUn cuar...       male   argentina
3  ¡El diario de Daya. está disponible!  #opinión...     female   venezuela
4  @jaheaga nuuuu mijo, yo no se que tanto le ven...       male    colombia

El shape de nuestro dataframe es:  (4200, 3)


In [ ]:
#ordenamos nuestros datos
######***  Set de entrenamiento y etiquetas de entrenamiento  ***######   
X_train=trainDF['text'].values
y_train_gen=trainDF['labels_gen']
y_train_leng = trainDF['labels_leng']

######*** Hacemos el encoder de los paises ***#######
encoderG = preprocessing.LabelEncoder()
encoderL = preprocessing.LabelEncoder()
y_trainGen = encoderG.fit_transform(y_train_gen)
y_trainLen = encoderL.fit_transform(y_train_leng)

print("Estas son las etiquetas: ",np.unique(y_trainGen),"\nY estos el significado: ", encoderG.inverse_transform(np.unique(y_trainGen)))
print("Estas son las etiquetas: ",np.unique(y_trainLen),"\nY estos el significado: ", encoderL.inverse_transform(np.unique(y_trainLen)))

Estas son las etiquetas:  [0 1] 
Y estos el significado:  ['female' 'male']
Estas son las etiquetas:  [0 1 2 3 4 5 6] 
Y estos el significado:  ['argentina' 'chile' 'colombia' 'mexico' 'peru' 'spain' 'venezuela']


In [ ]:
#checamos si tenemos disponible GPU, si es así lo usamos
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
#importamos modelo
import transformers as ppb
#utilizamos la version base y multilingual de BERT
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-multilingual-cased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# el modelo debe de entregar todos los hidden states para poder usarlos
model = model_class.from_pretrained(pretrained_weights,output_hidden_states=True)

In [ ]:
#checamos como tokeniza una sola oración
print(' Original: ', X_train[0])
# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(X_train[0]))
# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(X_train[0])))

 Original:  ¡Recursos Didácticos Interactivos está disponible!  Gracias a @tadvisorsguild @GDataES @Stagraphik #educación
¡Pigmalión Innovación Nticx está disponible!  Gracias a @LeibmanCalidad @DrLopezRosetti @ElenSalin #educación
¡Recursos Didácticos Interactivos está disponible!  Gracias a @advaldepenas @SocialMediaTIC_ #educación #educacion
¡Pigmalión Innovación Nticx está disponible!  Gracias a @66SantosMariana @profetecnolorca @jeanmolinafarin #educación
¡Recursos Didácticos Interactivos está disponible!  Gracias a @nelidacandia4 @conectaline #educación #educacion
¡Pigmalión Innovación Nticx está disponible!  Gracias a @historiatotal @YOLANDA_EGEA @aljisa2010 #educación
Me ha gustado un vídeo de @YouTube de @apixelados ( - UN ESTANCADO TROLLEA A SU MADRE | MINECRAFT MAPA TROLL
Tuiter on buenos días, Creer posible algo es hacerlo cierto.  Friedrich Hebbel
¡Recursos Didácticos Interactivos está disponible!  Gracias a @yolandatarango @ximoZorrilla11 @gfigols #educación
¡Pigmalión In

In [ ]:
#determinamos el tamaño máximo de nuestros textos
#el máximo de BERT es 512
print("Máximo de Tokens de BERT: ",tokenizer.model_max_length)

Máximo de Tokens de BERT:  512


In [ ]:
#tokenizamos todas nuestras oraciones y los convertimos a tensores de pytorch
input_ids = []
attention_masks = []

# For every sentence...
contador = 0
for sent in X_train:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = tokenizer.model_max_length,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
    if contador%100 == 0:
      print(contador)
    contador +=1

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

#vemos un ejemplo
print('Original: ', X_train[0])
print('Token IDs:', input_ids[0])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
Original:  ¡Recursos Didácticos Interactivos está disponible!  Gracias a @tadvisorsguild @GDataES @Stagraphik #educación
¡Pigmalión Innovación Nticx está disponible!  Gracias a @LeibmanCalidad @DrLopezRosetti @ElenSalin #educación
¡Recursos Didácticos Interactivos está disponible!  Gracias a @advaldepenas @SocialMediaTIC_ #educación #educacion
¡Pigmalión Innovación Nticx está disponible!  Gracias a @66SantosMariana @profetecnolorca @jeanmolinafarin #educación
¡Recursos Didácticos Interactivos está disponible!  Gracias a @nelidacandia4 @conectaline #educación #educacion
¡Pigmalión Innovación Nticx está disponible!  Gracias a @historiatotal @YOLANDA_EGEA @aljisa2010 #educación
Me ha gustado un vídeo de @YouTube de @apixelados ( - UN ESTANCADO TROLLEA A SU MADRE | MINECRAFT MAPA TROLL
Tuiter on

In [ ]:
#creamos un iterador para nuestra base, para ahorrar memoria
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset

batch_size = 32

prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
#corremos el modelo

#movemos el modelo a GPI
model.to(device)
#ponemos el modelo en modo de evaluación
model.eval()

#guardamos los estados de [CLS] de las ultimas cuatro capas
hiddenLayer12,hiddenLayer11,hiddenLayer10,hiddenLayer9 = [],[],[],[] 

contador = 0
#corremos por cada bach
for batch in prediction_dataloader:
  # guardamos en GPU
  batch = tuple(t.to(device) for t in batch)
  
  # obtenemos los datos
  texto, mask = batch
  #evaluamos con el modelo
  with torch.no_grad():
    last_hidden_states = model(texto, attention_mask=mask)
  #guardamos los outputs
  # de la capa -1, tomamos todas las oraciones, la palabra [CLS], todos los valores
  layer12 = (last_hidden_states[2][-1][:,0,:]).detach().cpu().numpy().tolist()
  layer11 = (last_hidden_states[2][-2][:,0,:]).detach().cpu().numpy().tolist()
  layer10 = (last_hidden_states[2][-3][:,0,:]).detach().cpu().numpy().tolist()
  layer9 = (last_hidden_states[2][-4][:,0,:]).detach().cpu().numpy().tolist()

  #append
  hiddenLayer12.append(layer12)
  hiddenLayer11.append(layer11)
  hiddenLayer10.append(layer10)
  hiddenLayer9.append(layer9)

  if contador%10 ==0:
    print(contador)
    #print(last_hidden_states[2][-1].size())
  contador +=1
print('DONE.')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
DONE.


In [ ]:
#volvemos las lista de output en un shape correcto
from itertools import chain
hl12 = list(chain.from_iterable(hiddenLayer12))
hl11 = list(chain.from_iterable(hiddenLayer11))
hl10 = list(chain.from_iterable(hiddenLayer10))
hl9 = list(chain.from_iterable(hiddenLayer9))

#elige el output
#contatenado de las últimas cuatro capas
hlunion = [x+y+z+w for x,y,z,w in zip(hl12,hl11,hl10,hl9)]
#promedio de las ultimas capas
hlunion = [(np.sum([x,y,z,w], axis=0)/4).tolist() for x,y,z,w in zip(hl12,hl11,hl10,hl9)]

# tenemos nuestro array final
hiddenStates = np.array(hlunion)
print(hiddenStates.shape)

(4200, 768)


In [ ]:
#creamos los clasificadores

#incrementamos el maximo de iteraciones de ambos modelos por problemas de convergencia
# de 1,000 a 10,000
classifiers = []
classifiers.append(linear_model.LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False))
classifiers.append(svm.LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))


In [ ]:
#corremos SVM y regresión logistica con los datos
#Evaluación para ver el género de la persona
Evaluacion(classifiers, hiddenStates, y_trainGen, input_pathImp, tipo_datos, "Género")

#Evaluación para ver la variante de lenguaje de la persona
Evaluacion(classifiers, hiddenStates, y_trainLen, input_pathImp, tipo_datos, "Lenguaje")

*** Entrenando primer modelo *** 1
*** Predecir primer modelo *** 1
--- 8.948150873184204 seconds ---
*** Entrenando segundo modelo *** 1


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


*** Predecir segundo modelo *** 1
--- 66.48294043540955 seconds ---
*** Termino el entrenamiento de la capa  1  ***
--- 66.50140881538391 seconds ---
*** Entrenando primer modelo *** 2
*** Predecir primer modelo *** 2
--- 75.64027094841003 seconds ---
*** Entrenando segundo modelo *** 2


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


*** Predecir segundo modelo *** 2
--- 132.83861923217773 seconds ---
*** Termino el entrenamiento de la capa  2  ***
--- 132.85502004623413 seconds ---
*** Entrenando primer modelo *** 3
*** Predecir primer modelo *** 3
--- 142.26214957237244 seconds ---
*** Entrenando segundo modelo *** 3


...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


*** Predecir segundo modelo *** 23
--- 5313.1394720077515 seconds ---
*** Termino el entrenamiento de la capa  23  ***
--- 5313.156820774078 seconds ---
*** Entrenando primer modelo *** 24
*** Predecir primer modelo *** 24
--- 5350.06291103363 seconds ---
*** Entrenando segundo modelo *** 24


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


*** Predecir segundo modelo *** 24
--- 5541.91409444809 seconds ---
*** Termino el entrenamiento de la capa  24  ***
--- 5541.932635068893 seconds ---
*** Entrenando primer modelo *** 25
*** Predecir primer modelo *** 25
--- 5582.464325904846 seconds ---
*** Entrenando segundo modelo *** 25
*** Predecir segundo modelo *** 25
--- 5781.689937353134 seconds ---
*** Termino el entrenamiento de la capa  25  ***
--- 5781.709213256836 seconds ---


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
df = pd.DataFrame(hiddenStates)
df["Lenguaje"] = y_trainLen
df["Género"] = y_trainGen

#df.to_csv(input_pathImp+"/Bert es_clean base.csv", index=False, encoding="utf-8-sig")


Little Shi

shi shi shi 

